# Introduction to Docker

**Learning Objectives**
  * Build and run Docker containers
  * Pull Docker images from Docker Hub and Google Container Registry
  * Push Docker images to Google Container Registry

## Overview

Docker is an open platform for developing, shipping, and running applications. With Docker, you can separate your applications from your infrastructure and treat your infrastructure like a managed application. Docker helps you ship code faster, test faster, deploy faster, and shorten the cycle between writing code and running code.

Docker does this by combining kernel containerization features with workflows and tooling that helps you manage and deploy your applications.

Docker containers can be directly used in Kubernetes, which allows them to be run in the Kubernetes Engine with ease. After learning the essentials of Docker, you will have the skillset to start developing Kubernetes and containerized applications.

## Basic Docker commands

See what docker images you have. 

In [1]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED        SIZE
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   2 months ago   1.73GB


If this is the first time working with docker you won't have any repositories listed. 

**Note**. If you are running this in an AI Notebook, then you should see a single image `gcr.io/inverting-proxy/agent`. This is the container that is currently running the AI Notebook. 

Let's use `docker run` to pull a docker image called `hello-world` from the public registry. The docker daemon will search for the `hello-world` image, if it doesn't find the image locally, it pulls the image from a public registry called Docker Hub, creates a container from that image, and runs the container for you.

In [2]:
!docker run hello-world

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world

de127a29: Pull complete 529kB/2.529kBBDigest: sha256:f2266cbfc127c960fd30e76b7c792dc23b588c0db76233517e1891a4e357d519
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For 

Now when we look at our docker images we should see `hello-world` there as well.

In [3]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED        SIZE
hello-world                    latest    d1165f221234   7 weeks ago    13.3kB
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   2 months ago   1.73GB


This is the image pulled from the Docker Hub public registry. The Image ID is in `SHA256` hash format—this field specifies the Docker image that's been provisioned. When the docker daemon can't find an image locally, it will by default search the public registry for the image. Let's run the container again:

Now, if we want to run `docker run hello-world` again, it won't have to download from the container registry.

To see all docker containers running, use `docker ps`.

In [4]:
!docker ps

CONTAINER ID   IMAGE                          COMMAND                  CREATED          STATUS          PORTS     NAMES
facda91186f6   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   13 minutes ago   Up 13 minutes             proxy-agent


There are no running containers. **Note. If you are running this in at AI Notebook, you'll see one container running.**

The `hello-world` containers you ran previously already exited. In order to see all containers, including ones that have finished executing, run docker `ps -a`:

In [5]:
!docker ps -a

CONTAINER ID   IMAGE                          COMMAND                  CREATED              STATUS                          PORTS     NAMES
2500d9bbbeb1   hello-world                    "/hello"                 About a minute ago   Exited (0) About a minute ago             wonderful_poitras
facda91186f6   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   14 minutes ago       Up 14 minutes                             proxy-agent


This shows you the Container ID, a UUID generated by Docker to identify the container, and more metadata about the run. The container Names are also randomly generated but can be specified with docker run --name [container-name] hello-world.

## Build a Docker container

Let's build a Docker image that's based on a simple node application.

**Exercise**

Open the text file called `intro.docker` in the `dockerfiles` folder and complete the TODO there. 

Your dockerfile should have the following steps

 1. use `FROM` to inherit an official Node runtime as the parent image; e.g. node:6
 2. use `WORKDIR` to seet the working directory to /app
 3. use `ADD` to copy the current directory to the container at /app
 4. use `EXPOSE` to make the containers port 80 available to the outside world
 5. use `CMD` to run the command `node ./src/app.js`

This file instructs the Docker daemon on how to build your image.

The initial line specifies the base parent image, which in this case is the official Docker image for node version 6.
In the second, we set the working (current) directory of the container.
In the third, we add the current directory's contents (indicated by the "." ) into the container.
Then we expose the container's port so it can accept connections on that port and finally run the node command to start the application.

Check out the other [Docker command references](https://docs.docker.com/engine/reference/builder/#known-issues-run) to understand what each line does.

We're going to use this Docker container to run a simple node.js app. Have a look at `app.js`. This is a simple HTTP server that listens on port 80 and returns "Hello World."


Now let's build the image. Note again the "`.`", which means current directory so you need to run this command from within the directory that has the Dockerfile.

The `-t` is to name and tag an image with the `name:tag` syntax. The name of the image is `node-app` and the tag is `0.1`. The tag is highly recommended when building Docker images. If you don't specify a tag, the tag will default to latest and it becomes more difficult to distinguish newer images from older ones. Also notice how each line in the Dockerfile above results in intermediate container layers as the image is built.

**Exercise**

Use `docker build` to build the docker image at `dockerfiles/intro.docker`. Tag the image `node-app:0.1`. 

In [18]:
%%bash
docker build -t node-app:0.1 -f dockerfiles/intro.docker .

Sending build context to Docker daemon  111.6kB
Step 1/5 : FROM node:6
 ---> ab290b853066
Step 2/5 : WORKDIR /app
 ---> Using cache
 ---> 009bea20c9c7
Step 3/5 : ADD . /app
 ---> f64d4ded400b
Step 4/5 : EXPOSE 80
 ---> Running in 98f76ba4bd62
Removing intermediate container 98f76ba4bd62
 ---> e94b0c9d449f
Step 5/5 : CMD ["node", "./src/app.js"]
 ---> Running in 581263f30192
Removing intermediate container 581263f30192
 ---> 379879890130
Successfully built 379879890130
Successfully tagged node-app:0.1


Let's check that the image has been created correctly. 

In [19]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED         SIZE
node-app                       0.1       379879890130   8 seconds ago   884MB
<none>                         <none>    66685936760c   4 minutes ago   884MB
hello-world                    latest    d1165f221234   7 weeks ago     13.3kB
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   2 months ago    1.73GB
node                           6         ab290b853066   23 months ago   884MB


You should see a `node-app` repository that was created only seconds ago. 

Notice `node` is the base image and `node-app` is the image you built. You can't remove `node` without removing `node-app` first. The size of the image is relatively small compared to VMs. Other versions of the node image such as `node:slim` and `node:alpine` can give you even smaller images for easier portability. The topic of slimming down container sizes is further explored in Advanced Topics. You can view all versions in the official repository here.

Note, you can remove an image from your docker images using `docker rmi [repository]:[tag]`.

## Run a Docker container

Now we'll run the container based on the image you built above using the `docker run` command. The `--name` flag allows you to name the container if you like. And `-p` instructs Docker to map the host's port 4000 to the container's port 80. This allows you to reach the server at http://localhost:4000. Without port mapping, you would not be able to reach the container at localhost.

In [20]:
!docker ps -a

CONTAINER ID   IMAGE                          COMMAND                    CREATED              STATUS                            PORTS     NAMES
6a134be8fd43   66685936760c                   "/bin/sh -c '[\"node\"…"   About a minute ago   Exited (127) About a minute ago             mystifying_agnesi
b5cdc0cd4a72   66685936760c                   "/bin/sh -c '[\"node\"…"   3 minutes ago        Exited (127) 3 minutes ago                  hardcore_shirley
2500d9bbbeb1   hello-world                    "/hello"                   16 minutes ago       Exited (0) 16 minutes ago                   wonderful_poitras
facda91186f6   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"     29 minutes ago       Up 29 minutes                               proxy-agent


**Exercise**

Use `docker run` to run the container you just build called `node-app:0.1`. Assign the host port `4000` to port `80` and assign it the name `my-app`.

In [21]:
%%bash
docker run -p 4000:80 node-app:0.1

Server running at http://0.0.0.0:80/


CalledProcessError: Command 'b'docker run -p 4000:80 node-app:0.1\n'' returned non-zero exit status 137.

To test out the server, open a terminal window and type the following command:

```bash
curl http://localhost:4000
```

You should see the server respond with `Hello World`

The container will run as long as the initial terminal is running. If you want to stop the container, run the following command in the terminal to stop and remove the container:

```bash
docker stop my-app && docker rm my-app
```
After a few moments the container will stop. You should notice the cell above will complete execution.

#### Running the container in the background
If you want to the container to run in the background (not tied to the terminal's session), you need to specify the `-d` flag.
Now run the following command to start the container in the background

**Exercise**

Modify your command above with `-d` flag to run `my-app` in the background.

In [22]:
%%bash
docker run -p 4000:80 -d node-app:0.1

d6a67cb46511507e17a0ba9ae85b2df66335829ed2eba80511f43f0ff36482d1


Your container is now running in the background. You can check the status of your running container using `docker ps`

In [23]:
!docker ps

CONTAINER ID   IMAGE                          COMMAND                  CREATED          STATUS          PORTS                  NAMES
d6a67cb46511   node-app:0.1                   "node ./src/app.js"      18 seconds ago   Up 17 seconds   0.0.0.0:4000->80/tcp   nostalgic_galileo
facda91186f6   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   34 minutes ago   Up 34 minutes                          proxy-agent


Notice the container is running in the output of docker ps. You can look at the logs by executing `docker logs [container_id]`. 

In [28]:
# Note, your container id will be different
!docker logs d6a67cb46511

Server running at http://0.0.0.0:80/


You should see 
```bash
Server running at http://0.0.0.0:80/
```
If you want to follow the log's output as the container is running, use the `-f` option.

## Modify & Publish

Let's modify the application and push it to your Google Cloud Repository (gcr). After that you'll remove all local containers and images to simulate a fresh environment, and then pull and run your containers from gcr. This will demonstrate the portability of Docker containers.

### Edit `app.js`
Open the file `./src/app.js` with the text editor and replace "Hello World" with another string. Then build this new image. 

**Exercise**

After modifying the `app.js` file, use `docker build` to build a new container called `node-app:0.2` from the same docker file. 

In [29]:
%%bash
docker build -t node-app:0.2 -f dockerfiles/intro.docker .

Sending build context to Docker daemon  111.1kB
Step 1/5 : FROM node:6
 ---> ab290b853066
Step 2/5 : WORKDIR /app
 ---> Using cache
 ---> 009bea20c9c7
Step 3/5 : ADD . /app
 ---> c1c7fc943a15
Step 4/5 : EXPOSE 80
 ---> Running in f6dbad8a013a
Removing intermediate container f6dbad8a013a
 ---> c2756ed5bf0b
Step 5/5 : CMD ["node", "./src/app.js"]
 ---> Running in 97f60b4f0dbb
Removing intermediate container 97f60b4f0dbb
 ---> 037fd15d4996
Successfully built 037fd15d4996
Successfully tagged node-app:0.2


Notice in `Step 2` of the output we are using an existing cache layer. From `Step 3` and on, the layers are modified because we made a change in `app.js`.

Run another container with the new image version. Notice how we map the host's port 8000 instead of 80. We can't use host port 4000 because it's already in use. 

**Exercise**

Run this new container in the background using a different port and with the name `my-app-2`.

In [30]:
!docker ps -a

CONTAINER ID   IMAGE                          COMMAND                  CREATED          STATUS                      PORTS                  NAMES
d6a67cb46511   node-app:0.1                   "node ./src/app.js"      3 minutes ago    Up 3 minutes                0.0.0.0:4000->80/tcp   nostalgic_galileo
2500d9bbbeb1   hello-world                    "/hello"                 25 minutes ago   Exited (0) 25 minutes ago                          wonderful_poitras
facda91186f6   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   38 minutes ago   Up 38 minutes                                      proxy-agent


In [31]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED              SIZE
node-app                       0.2       037fd15d4996   About a minute ago   884MB
node-app                       0.1       379879890130   9 minutes ago        884MB
<none>                         <none>    66685936760c   14 minutes ago       884MB
hello-world                    latest    d1165f221234   7 weeks ago          13.3kB
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   2 months ago         1.73GB
node                           6         ab290b853066   23 months ago        884MB


In [32]:
%%bash
docker run -p 4002:80 -d node-app:0.2

1e3f11abf2f1c971814c1e891cde884470e3e856a41525185f1a0c736f663742


You can check that both container are running using `docker ps`.

In [33]:
!docker ps

CONTAINER ID   IMAGE                          COMMAND                  CREATED          STATUS          PORTS                  NAMES
1e3f11abf2f1   node-app:0.2                   "node ./src/app.js"      4 seconds ago    Up 3 seconds    0.0.0.0:4002->80/tcp   stupefied_rhodes
d6a67cb46511   node-app:0.1                   "node ./src/app.js"      4 minutes ago    Up 4 minutes    0.0.0.0:4000->80/tcp   nostalgic_galileo
facda91186f6   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   39 minutes ago   Up 38 minutes                          proxy-agent


And let's test boht containers using `curl` as before:

In [34]:
!curl http://localhost:4002

No hello for no-world!


In [35]:
!curl http://localhost:4000

Hello Wolrd!


Recall, to stop a container running, you can execute the following command either in a terminal or (because they are running in the background) in a cell in this notebook. 

In [37]:
!docker stop 1e3f11abf2f1

1e3f11abf2f1


In [38]:
!docker stop d6a67cb46511

d6a67cb46511


### Publish to gcr

Now you're going to push your image to the Google Container Registry (gcr). To push images to your private registry hosted by gcr, you need to tag the images with a registry name. The format is `[hostname]/[project-id]/[image]:[tag]`.

For gcr:

  * `[hostname]`= gcr.io
  * `[project-id]`= your project's ID
  * `[image]`= your image name
  * `[tag]`= any string tag of your choice. If unspecified, it defaults to "latest".

In [40]:
import os

PROJECT_ID = "qwiklabs-gcp-01-4098456db096" # REPLACE WITH YOUR PROJECT NAME

os.environ["PROJECT_ID"] = PROJECT_ID

Let's tag `node-app:0.2`.

In [39]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED          SIZE
node-app                       0.2       037fd15d4996   3 minutes ago    884MB
node-app                       0.1       379879890130   12 minutes ago   884MB
<none>                         <none>    66685936760c   16 minutes ago   884MB
hello-world                    latest    d1165f221234   7 weeks ago      13.3kB
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   2 months ago     1.73GB
node                           6         ab290b853066   23 months ago    884MB


**Exercise**

Tag the `node-app:0.2` image with a new image name conforming to the naming convention `gcr.io/[project-id]/[image]:[tag]`. Keep the image and tag names the same.

In [43]:
!echo $PROJECT_ID

qwiklabs-gcp-01-4098456db096


In [44]:
%%bash
docker tag node-app:0.2 gcr.io/$PROJECT_ID/node-app:0.2

Now when we list our docker images we should see this newly tagged repository.

In [45]:
!docker images

REPOSITORY                                     TAG       IMAGE ID       CREATED          SIZE
gcr.io/qwiklabs-gcp-01-4098456db096/node-app   0.2       037fd15d4996   8 minutes ago    884MB
node-app                                       0.2       037fd15d4996   8 minutes ago    884MB
node-app                                       0.1       379879890130   16 minutes ago   884MB
<none>                                         <none>    66685936760c   21 minutes ago   884MB
hello-world                                    latest    d1165f221234   7 weeks ago      13.3kB
gcr.io/inverting-proxy/agent                   <none>    fe507176d0e6   2 months ago     1.73GB
node                                           6         ab290b853066   23 months ago    884MB


Next, let's push this image to gcr.

**Exercise**

Push this new image to the gcr.

In [47]:
%%bash
docker push gcr.io/qwiklabs-gcp-01-4098456db096/node-app:0.2

The push refers to repository [gcr.io/qwiklabs-gcp-01-4098456db096/node-app]
3bf1c8563466: Preparing
1d0a5f4c019f: Preparing
f39151891503: Preparing
f1965d3c206f: Preparing
a27518e43e49: Preparing
910d7fd9e23e: Preparing
4230ff7f2288: Preparing
2c719774c1e1: Preparing
ec62f19bb3aa: Preparing
f94641f1fe1f: Preparing
910d7fd9e23e: Waiting
4230ff7f2288: Waiting
f94641f1fe1f: Waiting
2c719774c1e1: Waiting
ec62f19bb3aa: Waiting
f1965d3c206f: Layer already exists
a27518e43e49: Layer already exists
f39151891503: Layer already exists
910d7fd9e23e: Layer already exists
2c719774c1e1: Layer already exists
4230ff7f2288: Layer already exists
f94641f1fe1f: Layer already exists
ec62f19bb3aa: Layer already exists
3bf1c8563466: Pushed
1d0a5f4c019f: Pushed
0.2: digest: sha256:3958efeb0c0e911aa1a93a12e3c6ddc1855408fac63ba4d40f934e4b9fafedb1 size: 2424


Check that the image exists in `gcr` by visiting the image registry Cloud Console. You can navigate via the console to `Navigation menu > Container Registry` or visit the url from the cell below:

In [46]:
%%bash
echo "http://gcr.io/${PROJECT_ID}/node-app"

http://gcr.io/qwiklabs-gcp-01-4098456db096/node-app


### Test the published gcr image

Let's test this image. You could start a new VM, ssh into that VM, and install gcloud. For simplicity, we'll just remove all containers and images to simulate a fresh environment.

First, stop and remove all containers using `docker stop` and `docker rm`. **Be careful not to stop the container running this AI Notebook!**.

In [ ]:
!docker stop my-app && docker rm my-app

In [50]:
!docker rm 1e3f11abf2f1

1e3f11abf2f1


In [ ]:
!docker stop my-app-2 && docker rm my-app-2

In [51]:
!docker rm d6a67cb46511

d6a67cb46511


Now remove the docker images you've created above using `docker rmi`.

In [52]:
!docker images

REPOSITORY                                     TAG       IMAGE ID       CREATED          SIZE
node-app                                       0.2       037fd15d4996   11 minutes ago   884MB
gcr.io/qwiklabs-gcp-01-4098456db096/node-app   0.2       037fd15d4996   11 minutes ago   884MB
node-app                                       0.1       379879890130   19 minutes ago   884MB
<none>                                         <none>    66685936760c   24 minutes ago   884MB
hello-world                                    latest    d1165f221234   7 weeks ago      13.3kB
gcr.io/inverting-proxy/agent                   <none>    fe507176d0e6   2 months ago     1.73GB
node                                           6         ab290b853066   23 months ago    884MB


In [53]:
%%bash
docker rmi node-app:0.2
docker rmi gcr.io/${PROJECT_ID}/node-app:0.2
docker rmi node-app:0.1
docker rmi node:6 
docker rmi -f hello-world:latest

Untagged: node-app:0.2
Untagged: gcr.io/qwiklabs-gcp-01-4098456db096/node-app:0.2
Untagged: gcr.io/qwiklabs-gcp-01-4098456db096/node-app@sha256:3958efeb0c0e911aa1a93a12e3c6ddc1855408fac63ba4d40f934e4b9fafedb1
Deleted: sha256:037fd15d49965fc2d65d909f6ca02a3bee23ca71705803cbfe2a9084fbf6d7d7
Deleted: sha256:c2756ed5bf0bb7ae19d05b0887ad149b8d2b3c1ce76611da70d9499fb2d93403
Deleted: sha256:c1c7fc943a15786ead5441458e40f546bcacbcb1804d18fc9db5f8cdcb650d2f
Deleted: sha256:4721f7541e84280f811092d21a82df2cb15f7346a6d0daac56d9a24ff87e39e6
Untagged: node-app:0.1
Deleted: sha256:379879890130adfc97dc4d920c462a1dc6eae3726d4616d4cb390814c5aa387f
Deleted: sha256:e94b0c9d449fdfd5751d64c80abeaf4ae982d0086aba422cb310a01769ec180f
Deleted: sha256:f64d4ded400be31ab5498bdfd4e771393403499a942c3e39338728823e8e92e4
Deleted: sha256:26bc8b5bbf7d136bb90cdb1afadfedd762ce2cb686ae5acd3796fa65bff29881
Untagged: node:6
Untagged: node@sha256:e133e66ec3bfc98da0440e552f452e5cdf6413319d27a2db3b01ac4b319759b3
Untagged: hello-

Confirm all images are removed with `docker images`.

In [54]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED          SIZE
<none>                         <none>    66685936760c   24 minutes ago   884MB
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   2 months ago     1.73GB


In [55]:
!docker rmi 66685936760c

Deleted: sha256:66685936760cd9e8bf8ad0d9150a337d24f5ff5f658a9e5e163a6a3898040d05
Deleted: sha256:fad4cf7068577b04767304a22ec3528bfe037c382602610622d18410dd349eb7
Deleted: sha256:8878d9d5d60ad6986c117c73b19f67303f5d820c23e9d25cef72c51833f17124
Deleted: sha256:da4ed60304fd00eaa7e3455191f4011563241c0739f6ce5403e8885dc5662ce3
Deleted: sha256:009bea20c9c76ef246e243c89868113161cd81cd48878c822df08e020a241443
Deleted: sha256:73343327aaddb22fad6cb2bb8b4754ae07ae15e9f8de4b5a6975fa3aa7ec15fc
Deleted: sha256:ab290b853066caedc75778bd4839da568ecb2a6b8e27442e0b1ce3c4741f1184
Deleted: sha256:bf0467917e15808e6e9c38268a58f048842fefb22f0a28b86e822c3e15630b14
Deleted: sha256:dbc5d843fe6c57402d1d4b5cdded731b87189ea669351a96e1cb6ff5ffaa77a6
Deleted: sha256:8d8dabce32d508dfcd126ab43cde7b690764c4bd06800e7cac816d367473ad3e
Deleted: sha256:addff6454b75bcccc23ba0cdbf9ff5d83f6df71523cd3428261d1785c3e3c592
Deleted: sha256:dae4f674403b331cb8c19b2c238ffa7d5a7249a1b4fafc4725054201dce3341a
Deleted: sha256:a9a9c8853295

In [56]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED        SIZE
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   2 months ago   1.73GB


At this point you should have a pseudo-fresh environment. Now, pull the image and run it.

In [57]:
%%bash
docker pull gcr.io/${PROJECT_ID}/node-app:0.2
docker run -p 4000:80 -d gcr.io/${PROJECT_ID}/node-app:0.2

0.2: Pulling from qwiklabs-gcp-01-4098456db096/node-app
c5e155d5a1d1: Pulling fs layer
221d80d00ae9: Pulling fs layer
4250b3117dca: Pulling fs layer
3b7ca19181b2: Pulling fs layer
425d7b2a5bcc: Pulling fs layer
69df12c70287: Pulling fs layer
ea2f5386a42d: Pulling fs layer
d421d2b3c5eb: Pulling fs layer
1b812abe4a23: Pulling fs layer
ec50562eaa27: Pulling fs layer
3b7ca19181b2: Waiting
425d7b2a5bcc: Waiting
69df12c70287: Waiting
ea2f5386a42d: Waiting
d421d2b3c5eb: Waiting
1b812abe4a23: Waiting
ec50562eaa27: Waiting
4250b3117dca: Download complete
221d80d00ae9: Verifying Checksum
221d80d00ae9: Download complete
c5e155d5a1d1: Verifying Checksum
c5e155d5a1d1: Download complete
69df12c70287: Verifying Checksum
69df12c70287: Download complete
3b7ca19181b2: Verifying Checksum
3b7ca19181b2: Download complete
d421d2b3c5eb: Download complete
1b812abe4a23: Verifying Checksum
1b812abe4a23: Download complete
ec50562eaa27: Verifying Checksum
ec50562eaa27: Download complete
ea2f5386a42d: Verifying Ch

You can check that it's running as expected using before:

In [58]:
!curl http://localhost:4000

No hello for no-world!


Copyright 2020 Google LLC Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.